# Generic least squares function

This is a topical tutorial. If you are new to iminuit, you should go through the basic tutorial first. 

We have seen in the basic tutorial how to make a least-squares function with an explicit signature that iminuit could read to find the parameter names automatically. Part of the structure of a least-squares function is always the same. What changes is the model that predicts the y-values and its parameters.

Here we show how to make a generic least-squares class that works with iminuit.

Note: cost functions for least-squares and maximum likelihood fits can be imported from `iminuit.cost` 

In [1]:
import numpy as np
from iminuit import Minuit
from iminuit.util import describe, make_func_code
import traceback

In [2]:
class LeastSquares:
    """
    Generic least-squares cost function.
    """
    def __init__(self, model, x, y):
        self.model = model  # model predicts y for given x
        self.x = np.array(x)
        self.y = np.array(y)

    def __call__(self, *par):  # we accept a variable number of model parameters
        ym = self.model(self.x, *par)
        chi2 = np.sum((self.y - ym)**2)
        return chi2

Let's try it out with iminuit.

In [3]:
def line(x, a, b):  # simple straight line model with explicit parameters
    return a + b * x

x_data = [1, 2, 3, 4, 5]
y_data = [2, 4, 6, 8, 10]

lsq = LeastSquares(line, x_data, y_data)

# this fails
try:
    m = Minuit(lsq, pedantic=False)
except:
    traceback.print_exc()

Traceback (most recent call last):
  File "<ipython-input-3-b71ea06d0a85>", line 11, in <module>
    m = Minuit(lsq, pedantic=False)
  File "src/iminuit/_libiminuit.pyx", line 579, in iminuit._libiminuit.Minuit.__init__
  File "/Users/hdembinski/Extern/iminuit/src/iminuit/util.py", line 404, in describe
    raise TypeError("Unable to obtain function signature")
TypeError: Unable to obtain function signature


What happened? iminuit uses introspection to detect the parameter names and the number of parameters. It uses the  `describe` utility for that, but it fails, since the generic method signature `LeastSquares.__call__(self, *par)`, does not reveal the number and names of the parameters.

The information could be extracted from the model signature, but iminuit knows nothing about the signature of `line(x, a, b)`. We can fix this by generating a function signature for the `LeastSquares` class.

In [4]:
# get the args from line and strip 'x'
describe(line)[1:]

['a', 'b']

In [5]:
# now inject that into the lsq object with the make_func_code tool
lsq.func_code = make_func_code(describe(line)[1:])

# now we get the right answer
describe(lsq)

['a', 'b']

We can put this code into the init function of our generic least-squares class to obtain a generic least-squares class which works with iminuit.

In [6]:
class BetterLeastSquares(LeastSquares):
    def __init__(self, model, x, y):
        super().__init__(model, x, y)
        self.func_code = make_func_code(describe(model)[1:])

In [7]:
lsq = BetterLeastSquares(line, x_data, y_data)

In [8]:
m = Minuit(lsq, pedantic=False)
m.migrad()

------------------------------------------------------------------
| FCN = 6.421e-25               |      Ncalls=30 (30 total)      |
| EDM = 6.43e-25 (Goal: 0.0002) |            up = 1.0            |
------------------------------------------------------------------
|  Valid Min.   | Valid Param.  | Above EDM | Reached call limit |
------------------------------------------------------------------
|     True      |     True      |   False   |       False        |
------------------------------------------------------------------
| Hesse failed  |   Has cov.    | Accurate  | Pos. def. | Forced |
------------------------------------------------------------------
|     False     |     True      |   True    |   True    | False  |
------------------------------------------------------------------
------------------------------------------------------------------------------------------
|   | Name |   Value   | Hesse Err | Minos Err- | Minos Err+ | Limit-  | Limit+  | Fixed |
------------------------------------------------------------------------------------------
| 0 | a    |     0     |     1     |            |            |         |         |       |
| 1 | b    |   2.00    |   0.32    |            |            |         |         |       |
------------------------------------------------------------------------------------------

It works :).